## Initialization

### imports

Data download related

In [ ]:
import os
from os import path   # system path
from os import mkdir  # create a directory
from glob import glob # query folders
import requests       # download
import zipfile        # extract zip files
from tqdm import tqdm # progress bar

Data manipulation

In [ ]:
import geopandas as gpd # geo Pandas
import pandas as pd # Pandas
import matplotlib.pyplot as plt # to make graphs
import random # random
import numpy as np # numpy
from shapely.geometry import Point, Polygon, MultiPolygon, LineString, MultiLineString # Funzione per generare punti casuali all'interno di un poligono # to deal with geometries

### data

In [ ]:
COLAB = None
try:
    # only for google drive linking (on Colab)
    from google.colab import drive
    # Link Google Drive account to use relative pathnames
    drive.mount("/content/gdrive", readonly = False)
    COLAB = True
except:
    COLAB = False
print("Notbook is running in Colab environment:", COLAB)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Notbook is running in Colab environment: True


In [ ]:
#CSV_AND_JSON_FOLDER = "/content/gdrive/MyDrive/Data Science/smart cities/CSVs and JSONs from notebooks" # Edo Colab
CSV_AND_JSON_FOLDER = "/content/gdrive/MyDrive/smart cities/CSVs and JSONs from notebooks" # Davide Colab

In [ ]:
# Elenca tutti i file nella cartella
files = os.listdir(CSV_AND_JSON_FOLDER)

# Filtra solo i file CSV
csv_files = [f for f in files if f.endswith('csv')]

# Crea due dizionari per memorizzare i DataFrame
pd_dataframes = {}
gpd_dataframes = {}

# Leggi ogni file CSV in un DataFrame
for csv_file in csv_files:
    file_path = os.path.join(CSV_AND_JSON_FOLDER, csv_file)
    pd_dataframes[csv_file] = pd.read_csv(file_path)
    print(f"Imported {csv_file}")

# Filtra solo i file JSON
json_files = [f for f in files if f.endswith('json')]

# Leggi ogni file JSON in un DataFrame
for json_file in json_files:
    file_path = os.path.join(CSV_AND_JSON_FOLDER, json_file)
    gpd_dataframes[json_file] = gpd.read_file(file_path)
    print(f"Imported {json_file}")

Imported green_divided_in_areas.csv
Imported buildings_divided_in_areas.csv
Imported air_milano.csv
Imported air_sc.csv
Imported green_points.json
Imported comune.json
Imported sc_stations.json
Imported stations_milano.json
Imported osm_milano_natural.json
Imported osm_milano_buildings.json


In [ ]:
comune = gpd_dataframes['comune.json']
comune.head()

In [ ]:
stations_milano = gpd_dataframes['stations_milano.json']
stations_milano.head()

In [ ]:
sc_stations = gpd_dataframes['sc_stations.json']
sc_stations.head()

In [ ]:
osm_milano_buildings = gpd_dataframes['osm_milano_buildings.json']
osm_milano_buildings.head()

In [ ]:
milano = comune[comune["COMUNE"] == "Milano"].geometry.iloc[0]

### functions

In [ ]:
def gdf_merge(gdfs):
  gdf_ref = gdfs[0]
  return pd.concat(
    [gdf.to_crs(gdf_ref.crs) for gdf in gdfs],
    ignore_index=True,
  )

In [ ]:
def gplot_milano(gdf, column = "dataset", **kwargs):
  if (type(gdf) is list):
    return gplot_milano(gdf_merge(gdf), column = column, **kwargs)
  plot_gdf = gpd.GeoDataFrame(
    pd.concat([
      comune[comune["COMUNE"] == "Milano"],
      gdf.to_crs(comune.crs),
    ], ignore_index=True),
    crs=comune.crs,
  )
  return plot_gdf.plot(column = column, **kwargs)

## Data Augmentation

In [ ]:
stations_milano

In [ ]:
stations_milano = stations_milano.rename(columns={'nome': 'name', 'id_amat': 'id'})
stations_milano["id"] = stations_milano["id"].astype(str)

# aggiunte colonne id (necessaria per fare il join con i dati di rilevazione dell'aria) e dataset (per i plot)
sc_stations_filtered = sc_stations[['id', 'name', 'geometry', 'dataset']]
# eliminate le stazioni che non coprono il 2023, anno su cui abbiamo i dati delle rilevazioni
stations_milano_filtered = stations_milano[(stations_milano['inizio_operativita'] < '2023-01-01') & (stations_milano['fine_operativita'] >= '2024-01-01')]
stations_milano_filtered = stations_milano_filtered[['id', 'name', 'geometry', 'dataset']]

concatenated_stations = gdf_merge([sc_stations_filtered, stations_milano_filtered])

concatenated_stations['latitudine'] = concatenated_stations.geometry.y
concatenated_stations['longitudine'] = concatenated_stations.geometry.x

gplot_milano(concatenated_stations)
concatenated_stations

In [ ]:
# Calcola l'area di ciascuna geometria
osm_milano_buildings_non_punti = osm_milano_buildings[osm_milano_buildings.geometry.type != 'Point']
osm_milano_buildings_non_punti['area'] = osm_milano_buildings_non_punti.geometry.area

# Trova l'area minima
area_minima = osm_milano_buildings_non_punti['area'].min()
print("area minima", area_minima)

# Trova l'area media
area_media = osm_milano_buildings_non_punti['area'].mean()
print("area media", area_media)

# Trova il secondo percentile
area_secondo_percentile = osm_milano_buildings_non_punti['area'].quantile(0.02)
print("Secondo percentile dell'area:", area_secondo_percentile)

In [ ]:
# Funzione per generare punti casuali all'interno di un poligono
def random_points_in_polygon(polygon, num_points):
    min_x, min_y, max_x, max_y = polygon.bounds
    points = []
    while len(points) < num_points:
        random_point = Point(random.uniform(min_x, max_x), random.uniform(min_y, max_y))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

# Funzione per generare punti casuali lungo una linea
def random_points_in_linestring(linestring, num_points):
    points = []
    total_length = linestring.length
    for _ in range(num_points):
        random_distance = random.uniform(0, total_length)
        point = linestring.interpolate(random_distance)
        points.append(point)
    return points

# Funzione per generare punti in base all'area dei poligoni
def generate_points(df, points_per_sqm=1/area_secondo_percentile):
    points = []
    indices = []

    for idx, row in df.iterrows():
        if row['geometry'].geom_type == 'Polygon':
            coefficiente_altezza = 1+row['building:levels']//3
            area_sqm = row['geometry'].area # * 1e6  # Convertire km^2 in m^2
            num_points = max(int(area_sqm * points_per_sqm * coefficiente_altezza), 1)
            points.extend(random_points_in_polygon(row['geometry'], num_points))
            indices.extend([idx] * num_points)
        elif row['geometry'].geom_type == 'MultiPolygon':
            for polygon in row['geometry'].geoms:
                coefficiente_altezza = 1+row['building:levels']//3
                area_sqm = polygon.area # * 1e6  # Convertire km^2 in m^2
                num_points = max(int(area_sqm * points_per_sqm * coefficiente_altezza), 1)
                points.extend(random_points_in_polygon(polygon, num_points))
                indices.extend([idx] * num_points)
        elif row['geometry'].geom_type == 'Point':
            points.append(row['geometry'])
            indices.append(idx)
        else:
            print(f"warning: geometry type {row['geometry'].geom_type} not handled")

    return points, indices

In [ ]:
osm_milano_buildings['building:levels'].value_counts()

In [ ]:
building_levels = osm_milano_buildings['building:levels'].copy()
building_levels.fillna(0)

# Converti i valori numerici con `.5` al successivo intero
building_levels = building_levels.apply(lambda x: np.ceil(float(x)) if isinstance(x, str) and '.' in x else x)

# Converti i valori sopra 50 e le stringhe a 0
def convert_to_int(x):
    try:
        if pd.isna(x) or x == '' or x == 'piano terra':  # Gestisce i valori None, stringhe vuote e 'piano terra'
            return 0
        x = float(x)
        if x > 50:
            return 0
        return int(x)
    except ValueError:
        return 0

building_levels = building_levels.apply(convert_to_int)
osm_milano_buildings['building:levels'] = building_levels
osm_milano_buildings['building:levels'] = osm_milano_buildings['building:levels'].astype(int)

In [ ]:
# osm_milano_buildings_ristretto = osm_milano_buildings.head(59420)
# osm_milano_buildings_ristretto_2 = osm_milano_buildings.tail(1985) # dal 59476 in poi sono a posto
# osm_milano_buildings_ristretto_3 = osm_milano_buildings.iloc[59426:59445]
# osm_milano_buildings_ristretto_4 = osm_milano_buildings.iloc[59449:59465]
# osm_milano_buildings_ristretto_5 = osm_milano_buildings.iloc[59466:59475]
# osm_milano_buildings_ristretto_7 = osm_milano_buildings.iloc[[59465]]

# ho problemi al 24, al 45, al 48, al 65 e al 75


In [ ]:
indices = [59424, 59445, 59448, 59465, 59475] # gli ultimi due non hanno problemi, sono solo per il confronto
osm_milano_buildings_con_problemi = osm_milano_buildings.iloc[indices]
pd.set_option('display.max_colwidth', None)
osm_milano_buildings_con_problemi['geometry']

In [ ]:
pd.set_option('display.max_colwidth', 50)

In [ ]:
# Funzione per trasformare un poligono con punti ripetuti in un punto
def polygon_to_point(polygon):
    if isinstance(polygon, Polygon):
        coords = list(polygon.exterior.coords)
        #if len(set(coords)) == 1:  # Verifica se tutti i punti sono uguali
        return Point(coords[0])  # Usa il primo (e unico) punto come Point
    return polygon

# Applica la funzione per convertire i poligoni in punti
osm_milano_buildings_con_problemi['geometry'] = osm_milano_buildings_con_problemi['geometry'].apply(polygon_to_point)

# Verifica le modifiche
print(osm_milano_buildings_con_problemi['geometry'])

In [ ]:
osm_milano_buildings.loc[osm_milano_buildings_con_problemi.index, 'geometry'] = osm_milano_buildings_con_problemi['geometry']

In [ ]:
osm_milano_buildings.shape[0]

In [ ]:
# Funzione per ottenere i tipi di geometria e contarli
def get_geometry_types(df):
    return df['geometry'].apply(lambda x: x.geom_type).value_counts()

geometry_types = get_geometry_types(osm_milano_buildings)
print("\nTipi di geometria per 'buildings':")
print(geometry_types)

In [ ]:
# Genera punti per buildings

buildings_points, indices = generate_points(osm_milano_buildings)

points_df = pd.DataFrame({
    'geometry': buildings_points,
    'index': indices
})

buildings_points = osm_milano_buildings.loc[points_df['index']].reset_index(drop=True)
buildings_points['geometry'] = points_df['geometry'].values

# Ricava lat e lon dai punti generati
buildings_points['lat'] = buildings_points.geometry.y
buildings_points['lon'] = buildings_points.geometry.x

buildings_points = buildings_points[['lat', 'lon', 'geometry', 'dataset']]
buildings_points

per i poligoni e multipoligoni, io calcolerei l'area e dividerei poi il poligono in un numero di punti proporzionale all'area posti casualmente all'interno del poligono
esempio: parco nord ha un'area di 2km^2, allora avrò tanti punti classificati come grassland (tipo boh 2000) disposti casualmente all'interno della sua area.
l'aiuola piccolina sotto casa invece magari è 2mq quindi la posso considerare come un punto solo, un parchetto piccolino magari vale 100 punti etc
indicativamente magari un punto ogni 3mq, non so

In [ ]:
from geopy.distance import distance

# Distanza in metri
def calcola_distanza(punto1, punto2):
    return distance(punto1, punto2).meters

# Angolo in gradi
def calcola_angolo(point, reference):
  # point and reference are tuples (y, x) or (latitude, longitude)
  # must have the same CRS
  return np.arctan2(point[1] - reference[1], point[0] - reference[0]) * (180 / np.pi)

# Raggi in metri
# aggiungerei raggi più piccoli e diminuirei quelli più grossi
raggi = [100, 200, 500, 1000, 2000]

# da distanza a raggio
def calcola_raggio(distanza):
  for raggio in raggi:
    if distanza < raggio:
      return raggio
  return np.infty

settori = [1, 2, 3, 4, 5, 6, 7, 8]

# Divido gli angoli in 8
def calcola_settore(angolo):
    angolo_norm = angolo % 360
    settore = int(np.floor(angolo_norm / 45))  + 1
    return settore

In [ ]:
concatenated_stations

In [ ]:
def divide_in_areas(stazione_geometry, gdf):
  return_gdf = gdf.copy()
  return_gdf["distance"] = return_gdf["geometry"].apply(lambda x: gpd.GeoSeries([x.centroid], crs="epsg:32632").distance(stazione_geometry))
  return_gdf["raggio"] = return_gdf["distance"].apply(calcola_raggio).astype(str)
  # Calcolo dell'angolo rispetto alla direzione dell'est per ciascun albero
  # x is longitude
  # y is latitude
  return_gdf["angolo"] = np.arctan2(return_gdf.centroid.x - stazione_geometry.x, return_gdf.centroid.y - stazione_geometry.y) * (180 / np.pi)
  return_gdf["settore"] = return_gdf["angolo"].apply(calcola_settore).astype(str)
  return_gdf["area"] = "-"
  return_gdf["area"] = return_gdf["raggio"] + return_gdf["area"] + return_gdf["settore"]
  return return_gdf

In [ ]:
osm_milano_buildings.crs

In [ ]:
concatenated_stations.to_crs(32632)

In [ ]:
tmp_2 = divide_in_areas(concatenated_stations[concatenated_stations["name"] == "Piazza Leonardo"].geometry.iloc[0], osm_milano_buildings)

tmp_2["area"].value_counts()

In [ ]:
tmp_2.columns

In [ ]:
gplot_milano(tmp_2, column = "area", figsize=(13,13), markersize=0.05, cmap = "prism")

In [ ]:
tmp_3 = divide_in_areas(concatenated_stations[concatenated_stations["name"] == "Piazza Leonardo"].geometry.iloc[0], buildings_points)

tmp_3["area"].value_counts()

In [ ]:
gplot_milano(tmp_3, column = "area", figsize=(13,13), markersize=0.05, cmap = "prism")

In [ ]:
# Calcolo delle distanze e divisione per settori per ogni stazione
colonne = ['geometry', 'distance', 'raggio', 'angolo', 'settore', 'area']
risultati = gpd.GeoDataFrame(columns=['stazione'] + colonne, crs = "EPSG:32632")
for idx, stazione in concatenated_stations.iterrows():
    tmp = divide_in_areas(stazione.geometry, buildings_points)
    tmp = tmp[colonne]
    tmp['stazione'] = stazione['id']
    risultati = gpd.GeoDataFrame(pd.concat([risultati, tmp]), crs = "EPSG:32632")
risultati

In [ ]:
risultati_centro = risultati[risultati['stazione']=='Centro']
risultati_centro

## Export data

In [ ]:
file_path = os.path.join(CSV_AND_JSON_FOLDER, 'buildings_points.feather')
buildings_points.to_feather(file_path, index=False)

In [ ]:
file_path = os.path.join(CSV_AND_JSON_FOLDER, 'buildings_divided_in_areas.feather')
risultati.to_feather(file_path, index=False)